In [41]:
# IMPORTS
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.models import Input, Model
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, concatenate, Dense, Conv3D, MaxPool3D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 

In [4]:
# CONSTANTS
BSE_SHAPE = (500, 500, 1)
EBSD_SHAPE = (200, 200, 3)
OM_SHAPE = (1000, 1000, 1)


In [5]:
# VARIABLES
CONV_NODES = [64, 128, 256, 512]
DENSE_NODES = [32, 64, 128]

NUM_CONV = [0, 1]
NUM_DENSE = [0, 1, 2]


In [6]:
# TRAIN DATA IMPORT
BSE = 'path'
EBSD = 'path'
OM = 'path'


In [9]:
# #### testing network ####
test_path = 'test_data/'

# load test img
i = load_img('test_data/0/bse.png')
x = img_to_array(i)


In [52]:
# #### Build Model ####
# INPUTS
input_1 = Input(shape=(500,500,1))
input_2 = Input(shape=(500,500,1))
input_3 = Input(shape=(500,500,1))

# MODEL

# FIRST BRANCH
# BSE
# needs to accept a black and white image
one = Conv2D(64, (3, 3), activation="relu")(input_1)
one = MaxPool2D(pool_size=(2, 2))(one)

one = Conv2D(64, 3, activation='relu')(one)
one = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(one)

one = Model(inputs=input_1, outputs=one)

# SECOND BRANCH
# EBSD
# needs to accept a RGB image
two = Conv2D(64, (3, 3), activation='relu')(input_2)
two = MaxPool2D(pool_size=(2, 2))(two)

two = Conv2D(64, 3, activation='relu')(two)
two = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(two)

two = Model(inputs=input_2, outputs=two)

# THIRD BRANCH
# OM
# needs to accept a black and white image
three = Conv2D(64, (3, 3), activation='relu')(input_3)
three = MaxPool2D(pool_size=(2, 2))(three)

three = Conv2D(64, 3, activation='relu')(three)
three = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(three)

three = Model(inputs=input_3, outputs=three)

# COMBINE THE BRANCHES
# concatenates the three branches all together into one
combined = concatenate([one.output, two.output])
combined = concatenate([combined, three.output])

# END BRANCH
end = Dense(64, activation='relu')(combined)
end = Dense(1, activation='sigmoid')(end)

# MODEL
model = Model(inputs=[one.input, two.input, three.input], outputs=end)

# COMPILE MODEL
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])



In [54]:
# FIT MODEL
# callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                                      write_graph=True, write_images=True)

model.fit(validation_split=0.2, callbacks=[tensorboard_callback])
